# Lab 3 — ASR + LLM + TTS end-to-end

Este cuaderno implementa el flujo ASR → LLM → TTS solicitado en el Laboratorio 3. Incluye grabación de audio en Colab, transcripción con Whisper, generación de respuesta con un LLM local, síntesis con xTTS (coqui-tts) y un resumen de latencias por etapa.


## 0. Instalación de dependencias

Ejecuta esta celda una sola vez por sesión para instalar las librerías requeridas. Se utiliza `coqui-tts` (no el paquete `tts`) tal como solicita la guía.


In [ ]:
!pip install -q openai-whisper jiwer
!pip install -q transformers accelerate sentencepiece
!pip install -q coqui-tts


## 1. Grabación de audio en Colab

Se reutiliza el widget provisto por el profesor para capturar audio desde el navegador. Puedes volver a ejecutarlo cada vez que necesites una nueva muestra.


In [ ]:
from IPython.display import Javascript, HTML, display, Audio
from google.colab import output
import base64, subprocess, uuid, os
from pathlib import Path

# @title Widget de grabación compatible con Colab
def record(out_webm=None, out_wav=None, sr=16000, autoplay=False):
    """Graba audio desde el navegador y devuelve la ruta del WAV generado."""
    if out_webm is None:
        out_webm = f"/content/rec_{uuid.uuid4().hex}.webm"
    if out_wav is None:
        out_wav = f"/content/rec_{uuid.uuid4().hex}.wav"

    js = Javascript(r"""
    async function recorderUIOnce(){
      const existing = document.getElementById('recorder-box');
      if (existing) { existing.remove(); }

      const box = document.createElement('div');
      box.id = 'recorder-box';
      box.style.cssText = 'padding:12px;margin:8px 0;border:1px solid #ddd;border-radius:10px;display:inline-flex;gap:8px;align-items:center;font-family:sans-serif';

      const dot = document.createElement('span');
      dot.style.cssText = 'width:10px;height:10px;border-radius:50%;background:#bbb';

      const startBtn = document.createElement('button');
      startBtn.textContent = 'Grabar';
      startBtn.style.cssText = 'padding:6px 10px';

      const stopBtn = document.createElement('button');
      stopBtn.textContent = 'Parar';
      stopBtn.style.cssText = 'padding:6px 10px';
      stopBtn.disabled = true;

      const msg = document.createElement('span');
      msg.textContent = 'Listo para grabar';
      msg.style.minWidth = '180px';

      box.append(dot, startBtn, stopBtn, msg);
      document.body.appendChild(box);

      let stream, rec, chunks = [];

      function setRec(on){
        dot.style.background = on ? '#e74c3c' : '#bbb';
        startBtn.disabled = on;
        stopBtn.disabled = !on;
        msg.textContent = on ? 'Grabando…' : 'Listo para grabar';
      }

      return await new Promise(async (resolve, reject) => {
        try {
          stream = await navigator.mediaDevices.getUserMedia({ audio:true });
          rec = new MediaRecorder(stream);
        } catch (err) {
          box.remove();
          reject('No se pudo acceder al micrófono: ' + err);
          return;
        }

        rec.ondataavailable = e => { if (e.data && e.data.size > 0) chunks.push(e.data); };

        rec.onstop = async () => {
          try {
            const blob = new Blob(chunks, {type:'audio/webm;codecs=opus'});
            const buf = await blob.arrayBuffer();
            const b64 = btoa(String.fromCharCode(...new Uint8Array(buf)));
            stream.getTracks().forEach(t => t.stop());
            box.remove();
            resolve(b64);
          } catch (err) {
            stream.getTracks().forEach(t => t.stop());
            box.remove();
            reject(err);
          }
        };

        startBtn.onclick = () => { chunks = []; rec.start(); setRec(true); };
        stopBtn.onclick = () => {
          if (rec && rec.state === 'recording') {
            rec.stop();
            setRec(false);
          }
        };
      });
    }
    """)
    display(js)

    b64 = output.eval_js("recorderUIOnce()")
    with open(out_webm, 'wb') as f:
        f.write(base64.b64decode(b64))

    subprocess.run([
        'ffmpeg', '-y', '-i', out_webm, '-ac', '1', '-ar', str(sr), out_wav
    ], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    try:
        os.remove(out_webm)
    except FileNotFoundError:
        pass

    display(Audio(filename=out_wav, autoplay=autoplay))
    return out_wav


In [ ]:
# @title Graba la pregunta para el pipeline
QUESTION_WAV = record(out_wav='/content/input_question.wav', autoplay=True)
print('Pregunta guardada en:', QUESTION_WAV)


### Muestra de voz para clonación (opcional)

Graba o carga una muestra de voz (5–10 segundos) que represente al hablante. Esta muestra se utilizará como `speaker_wav` para xTTS y comparar frente a la voz base.


In [ ]:
# @title (Opcional) Graba la voz de referencia para clonación
# @markdown Ejecuta esta celda si deseas capturar una voz de referencia directamente en Colab.
CLONE_REFERENCE_WAV = record(out_wav='/content/voice_clone_ref.wav', autoplay=True)
print('Voz de referencia guardada en:', CLONE_REFERENCE_WAV)


In [ ]:
# Alternativa: sube un archivo WAV/MP3 existente desde tu computadora
# from google.colab import files
# uploaded = files.upload()
# CLONE_REFERENCE_WAV = next(iter(uploaded.keys()))  # usa la primera clave subida
# print('Archivo de referencia cargado:', CLONE_REFERENCE_WAV)

# Si ya conoces la ruta manualmente, puedes asignarla así:
# CLONE_REFERENCE_WAV = '/content/mi_referencia.wav'


## 2. Automatic Speech Recognition (ASR)

Usamos Whisper para transcribir el audio grabado. Se reporta la latencia y el texto reconocido.


In [ ]:
import gc
import time
import torch
import whisper

gc.collect()
torch.cuda.empty_cache()

ASR_MODEL_NAME = 'turbo'  # puedes cambiarlo por tiny, base, small, medium o large
whisper_model = whisper.load_model(ASR_MODEL_NAME)
print('Modelo Whisper cargado:', ASR_MODEL_NAME)


In [ ]:
def transcribe_audio(audio_path, *, language='es', task='transcribe', model=whisper_model):
    if not audio_path:
        raise ValueError('Debes proporcionar la ruta del audio a transcribir.')
    start = time.perf_counter()
    result = model.transcribe(audio_path, language=language, task=task, verbose=False)
    latency = time.perf_counter() - start
    text = result['text'].strip()
    return text, latency, result

TRANSCRIPT_TEXT, ASR_LATENCY, ASR_RAW = transcribe_audio(QUESTION_WAV, language='es')
print('ASR:', TRANSCRIPT_TEXT)
print(f'Latencia ASR: {ASR_LATENCY:.2f} s')


## 3. LLM — Generación de respuesta corta

Se utiliza FLAN-T5 (puedes cambiarlo por otro modelo disponible en HuggingFace). Se mide la latencia y se limita la respuesta a pocas oraciones.


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

LLM_MODEL_ID = 'google/flan-t5-base'
tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL_ID)
llm_model = AutoModelForSeq2SeqLM.from_pretrained(LLM_MODEL_ID, device_map='auto')
text_generator = pipeline('text2text-generation', model=llm_model, tokenizer=tokenizer)
print('Modelo LLM cargado:', LLM_MODEL_ID)


In [ ]:
PROMPT_TEMPLATE = (
    'Eres un asistente útil. Responde en 1-2 oraciones y sé directo. '
    'Pregunta: {transcript}'
)

def generate_reply(transcript, *, max_new_tokens=128):
    prompt = PROMPT_TEMPLATE.format(transcript=transcript)
    start = time.perf_counter()
    output = text_generator(prompt, max_new_tokens=max_new_tokens)
    latency = time.perf_counter() - start
    reply = output[0]['generated_text'].strip()
    return reply, latency, prompt

RESPONSE_TEXT, LLM_LATENCY, LAST_PROMPT = generate_reply(TRANSCRIPT_TEXT)
print('LLM:', RESPONSE_TEXT)
print(f'Latencia LLM: {LLM_LATENCY:.2f} s')
print('Prompt usado:', LAST_PROMPT)


## 4. TTS — Síntesis con Coqui TTS (xTTS)

Se carga el modelo `tts_models/multilingual/multi-dataset/xtts_v2`. Se sintetiza la respuesta con una voz base y, si se proporciona `CLONE_REFERENCE_WAV`, también con la voz clonada para comparar la calidad.


In [ ]:
from TTS.api import TTS

tts_model = TTS('tts_models/multilingual/multi-dataset/xtts_v2')
AVAILABLE_SPEAKERS = tts_model.speakers or []
print('Voces base disponibles:', AVAILABLE_SPEAKERS)
BASE_SPEAKER = AVAILABLE_SPEAKERS[0] if AVAILABLE_SPEAKERS else None
print('Voz base seleccionada:', BASE_SPEAKER)
LANGUAGE_CODE = 'es'  # cambia a 'en', 'pt', etc. si trabajas en otro idioma


In [ ]:
from IPython.display import Audio, display
import pandas as pd
from pathlib import Path
import time

def synthesize_with_xtts(text, *, output_path, speaker=None, speaker_wav=None, language='es'):
    start = time.perf_counter()
    kwargs = {
        'text': text,
        'language': language,
        'file_path': str(output_path),
    }
    if speaker:
        kwargs['speaker'] = speaker
    if speaker_wav:
        kwargs['speaker_wav'] = speaker_wav
    tts_model.tts_to_file(**kwargs)
    return time.perf_counter() - start

BASE_TTS_PATH = Path('/content/respuesta_base.wav')
CLONE_TTS_PATH = Path('/content/respuesta_clonada.wav')

print('Rutas de salida configuradas:')
print('  Base:', BASE_TTS_PATH)
print('  Clonada:', CLONE_TTS_PATH)


## 5. Pipeline integrado y métricas

La siguiente función ejecuta el pipeline completo (ASR → LLM → TTS), guarda los audios generados y muestra las latencias por etapa junto con el tiempo total.


In [ ]:
def run_pipeline(audio_path, *, asr_language='es', tts_language='es', base_speaker=BASE_SPEAKER, speaker_wav=None, max_new_tokens=128):
    if not audio_path:
        raise ValueError('Debes grabar o proporcionar un audio de entrada (QUESTION_WAV).')
    summary = []
    overall_start = time.perf_counter()

    transcript, asr_latency, _ = transcribe_audio(audio_path, language=asr_language)
    summary.append({'Etapa': 'ASR (Whisper)', 'Latencia (s)': asr_latency, 'Detalle': f'{len(transcript.split())} palabras'})
    print('Transcripción:', transcript)

    reply, llm_latency, prompt = generate_reply(transcript, max_new_tokens=max_new_tokens)
    summary.append({'Etapa': 'LLM (FLAN-T5)', 'Latencia (s)': llm_latency, 'Detalle': f'{len(reply.split())} palabras'})
    print('Respuesta generada:', reply)

    print('
▶️ Síntesis con voz base')
    base_latency = synthesize_with_xtts(reply, output_path=BASE_TTS_PATH, speaker=base_speaker, language=tts_language)
    summary.append({'Etapa': 'TTS voz base', 'Latencia (s)': base_latency, 'Detalle': base_speaker or 'default'})
    display(Audio(filename=str(BASE_TTS_PATH), autoplay=False))

    clone_path = None
    if speaker_wav and Path(speaker_wav).exists():
        print('
🎯 Síntesis con voz clonada')
        clone_latency = synthesize_with_xtts(reply, output_path=CLONE_TTS_PATH, speaker_wav=speaker_wav, language=tts_language)
        summary.append({'Etapa': 'TTS voz clonada', 'Latencia (s)': clone_latency, 'Detalle': Path(speaker_wav).name})
        clone_path = str(CLONE_TTS_PATH)
        display(Audio(filename=clone_path, autoplay=False))
    else:
        print('
No se encontró speaker_wav; se omite la comparación de voz clonada.')

    total_latency = time.perf_counter() - overall_start
    summary.append({'Etapa': 'Total pipeline', 'Latencia (s)': total_latency, 'Detalle': ''})

    metrics_df = pd.DataFrame(summary)
    display(metrics_df)

    return {
        'transcript': transcript,
        'reply': reply,
        'prompt': prompt,
        'metrics': metrics_df,
        'base_audio': str(BASE_TTS_PATH),
        'clone_audio': clone_path,
    }


In [ ]:
clone_wav = None
if 'CLONE_REFERENCE_WAV' in globals():
    clone_wav = CLONE_REFERENCE_WAV

results = run_pipeline(
    QUESTION_WAV,
    asr_language='es',
    tts_language=LANGUAGE_CODE,
    base_speaker=BASE_SPEAKER,
    speaker_wav=clone_wav,
    max_new_tokens=96,
)
results
